In [ ]:
#Shows Wemby's best performances in every statistical category for the season with opponent and date
stats_columns = ['Points', 'Rebounds', 'Assists', 'Blocks', 'Steals']

season_highs_list = []

for stat in stats_columns:
    max_index = df[stat].idxmax()
    
    max_value = df.loc[max_index, stat]
    max_opponent = df.loc[max_index, 'Opponent']
    max_date = df.loc[max_index, 'Date']

    season_highs_list.append({
        'Statistic': stat,
        'Season High': max_value,
        'Opponent': max_opponent,
        'Date': max_date
    })

print("-----------     Wemby's Season highs     -----------")
print(pd.DataFrame(season_highs_list))

     -----      Season highs      -----
  Statistic  Season High Opponent       Date
0    Points           40      DAL 2025-10-22
1  Rebounds           18      MIA 2025-10-31
2   Assists            6      MIA 2025-10-31
3    Blocks            9      NOP 2025-10-24
4    Steals            3      BKN 2025-10-26


In [ ]:
#Shows Wemby's performances based on his team's results
df_long_performances = df_long_final[df_long_final['Statistic'] != 'Minutes']

plt.figure(figsize=(12, 7))

sns.barplot(x='Statistic', y='Average', hue='Result', data=df_long_performances, 
            palette={'Win': "#2FB50A", 'Lose': "#C20909"},
            hue_order=['Win', 'Lose'])

for p in plt.gca().patches:
    h = p.get_height()
    if not np.isfinite(h) or h <= 0:
        continue
    plt.gca().annotate(f"{h:.2f}", 
                       (p.get_x() + p.get_width() / 2., h), 
                       ha='center', va='center', 
                       xytext=(0, 10), 
                       textcoords='offset points', 
                       fontsize=10)

plt.title("Wemby's performance based on his team's results", fontsize=18, weight='bold')
plt.xlabel('Statistic', fontsize=12)
plt.ylabel('Average per game', fontsize=12)
plt.legend(title="Game's Result")
plt.ylim(0, df_long_performances['Average'].max() * 1.2)
plt.tight_layout()
plt.show()

In [ ]:
#Preparing Wemby's averages based on his team's results
if 'Win/Lose' in df.columns:
    df['Result'] = df['Win/Lose'].map({'W': 'Win', 'L': 'Lose'}).fillna(df['Win/Lose'])
else:
    df['Result'] = df.get('Result', df.iloc[:, 0])

colonnes_stats_viz = ['Points', 'Rebounds', 'Assists', 'Blocks', 'Steals', 'Minutes']

win_averages = df[df['Result'] == 'Win'][colonnes_stats_viz].mean().round(2)
lose_averages = df[df['Result'] == 'Lose'][colonnes_stats_viz].mean().round(2)

df_plot = pd.DataFrame({
    'Win': win_averages,
    'Lose': lose_averages
}).reset_index()

df_plot.columns = ['Statistic', 'Win', 'Lose']

df_long_final = df_plot.melt(id_vars='Statistic', 
                             var_name='Result', 
                             value_name='Average')

print("--- Prepared Data for visualization ---")
print(df_long_final.head(12))

In [ ]:
#5 main statistics categories averages graph
visible_stats_averages = average[['Points', 'Rebounds', 'Assists', 'Blocks', 'Steals']].round(2)

df_averages = visible_stats_averages.reset_index()
df_averages.columns = ['Stat', 'Val']

plt.figure(figsize=(10, 6))

sns.barplot(x='Stat', y='Val', data=df_averages, color='#4B4A4A')

for index, row in df_averages.iterrows():
    plt.text(index, row['Val'] + 0.2, f"{row['Val']}", 
             color='#4B4A4A', ha="center", va='bottom', fontsize=12, weight='bold')

plt.title("Wemby's averages 25/26", fontsize=16, weight='bold')
plt.xlabel('', fontsize=12)
plt.ylabel('', fontsize=12)
plt.ylim(0, 35)

plt.tight_layout()
plt.show()

In [ ]:
columns_stats = ['Points', 'Rebounds', 'Assists', 'Blocks', 'Steals', 'Minutes'] 

average = df[columns_stats].mean()

# --- PERFORMANCE DETECTION ---

# "5x5" means that Wemby got at least 5 in all 5 main statistic categories

df['5x5'] = ( (df['Points'] >= 5) & 
              (df['Rebounds'] >= 5) & 
              (df['Assists'] >= 5) & 
              (df['Blocks'] >= 5) &
              (df['Steals'] >= 5) 
            ).astype(int)

#"Double Double" means that Wemby got at least 10 in 2 of the 5 main statistic categories

df['Double_Double'] = (
    ((df['Points'] >= 10) & (df['Rebounds'] >= 10)) |
    ((df['Points'] >= 10) & (df['Assists'] >= 10)) |
    ((df['Points'] >= 10) & (df['Blocks'] >= 10)) |
    ((df['Points'] >= 10) & (df['Steals'] >= 10)) |
    ((df['Rebounds'] >= 10) & (df['Assists'] >= 10)) |
    ((df['Rebounds'] >= 10) & (df['Blocks'] >= 10)) |
    ((df['Rebounds'] >= 10) & (df['Steals'] >= 10)) |
    ((df['Assists'] >= 10) & (df['Blocks'] >= 10)) |
    ((df['Assists'] >= 10) & (df['Steals'] >= 10)) |
    ((df['Blocks'] >= 10) & (df['Steals'] >= 10)) 
).astype(int)

conditions_td = [
    df['Points'] >= 10, 
    df['Rebounds'] >= 10, 
    df['Assists'] >= 10, 
    df['Blocks'] >= 10, 
    df['Steals'] >= 10
]
tens_count = sum(conditions_td)

#"Triple Double" means that Wemby got at least 10 in 3 of the 5 main statistic categories

df['Triple_Double'] = (tens_count >= 3).astype(int)

print("** Average statistics **")
print(average.round(2))
print("-" * 55)
print(f"Triple Doubles : {df['Triple_Double'].sum()}")
print(f"Double Doubles : {df['Double_Double'].sum()}")
print(f"5x5s : {df['5x5'].sum()}")

In [ ]:
#Points per Game evolution graph
plt.figure(figsize=(12, 6))

sns.lineplot(x='Date', y='Points', data=df, marker='o', 
             label='Points/Game', color='#4B4A4A', linewidth=3, markersize=10)

plt.ylim(0, 50)

for i in range(len(df)):
    plt.text(df['Date'][i], df['Points'][i] + 1, f"{df['Points'][i]}", 
             ha='center', fontsize=10, color="#4B4A4A", weight='bold')

plt.title('Points/Game Evolution 25/26', fontsize=14, weight='bold')
plt.xlabel('', fontsize=12)
plt.ylabel('', fontsize=12)

plt.legend(fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
df = pd.read_csv('wemby_25_26.csv')

df['Date'] = pd.to_datetime(df['Date']) 

colonnes_stats = ['Points', 'Rebounds', 'Assists', 'Blocks', 'Steals', 'Minutes']

for col in colonnes_stats:
    df[col] = pd.to_numeric(df[col], errors='coerce') 

print("Cleaned DataFrame :")
print(df.head())

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np